# **Introduction**

This notebook is used for testing and visualizing the performance of DQN-Agents by loading the weights of the Q-Network. The environment that is used for testing is the ``cartpole`` environment offered through Gymnasium.

# **Import Packages**

This section imports the necessary packages.

In [1]:
# import these packages:
import gymnasium as gym
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

from collections import deque